In [1]:
import os
%pwd
os.chdir("../")
%pwd

'e:\\ObjectDetection\\ChessPieceDetection'

In [2]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen= True)
class DataPreparationConfig:
    root_dir: Path
    data_dir: Path
    train_dir: Path
    test_dir: Path
    val_dir: Path
    data_yaml: Path

In [3]:
from src.CHESS_PIECE_DETECTION.constants import *
from src.CHESS_PIECE_DETECTION.utils.common import read_yaml, create_directories

In [4]:
class ConfigureationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_preparation_config(self) -> DataPreparationConfig:
        config = self.config.data_preparation

        create_directories([config.root_dir])

        data_preparation_config = DataPreparationConfig(
            root_dir = config.root_dir,
            data_dir = config.data_dir,
            train_dir = config.train_dir,
            test_dir = config.test_dir,
            val_dir = config.val_dir,
            data_yaml = config.data_yaml
        )

        return data_preparation_config

In [5]:
import shutil
import os
import yaml
from src.CHESS_PIECE_DETECTION import logger

In [6]:
class DataPreparation:
    def __init__(self, config):
        self.config = config
    
    def copying_data(self, train_path, test_path, val_path):
        train_folder = self.config.train_dir
        test_folder = self.config.test_dir
        val_folder = self.config.val_dir
        try:
            os.makedirs(train_folder, exist_ok=True)
            os.makedirs(test_folder, exist_ok=True)
            os.makedirs(val_folder, exist_ok=True)
            logger.info("Created validation, test, and train folders")
        except Exception as e:
            raise e

        paths = [train_folder, test_folder, val_folder]
        source = [train_path, test_path, val_path]
        for path, src in zip(paths, source):
            try:
                if not os.path.exists(path):
                    os.makedirs(path)
                shutil.copytree(src, path, dirs_exist_ok=True)
                logger.info(f"Folder copied successfully from {src} to {path}")
            except Exception as e:
                logger.info(f"Error occurred while copying folder: {e}")
    def change_yaml(self, data):
        train_folder = self.config.train_dir
        test_folder = self.config.test_dir
        val_folder = self.config.val_dir

        try:
            with open(data, 'r') as file:
                config_data = yaml.safe_load(file)

            config_data['train'] = train_folder+f'/images'
            config_data['val'] = val_folder+f'/images'
            config_data['test'] = test_folder+f'/images'

            with open(data, 'w') as file:
                yaml.safe_dump(config_data, file)

            logger.info("YAML file updated successfully.")

            data_yaml = self.config.data_yaml
            os.makedirs(self.config.root_dir, exist_ok=True)
            #dest_path = os.path.join(data_yaml)#, os.path.basename(data))
            shutil.copy(data, data_yaml)#, dest_path)
            logger.info(f"YAML file copied to {data_yaml}")

        except Exception as e:
            logger.info(f"Error while updating YAML file: {e}")

In [7]:
try:
    config = ConfigureationManager()
    data_preparation_config = config.get_data_preparation_config()
    data_preparation = DataPreparation(config=data_preparation_config)
    data_preparation.copying_data('datasets/artifacts/data_ingestion/train', 'datasets/artifacts/data_ingestion/test', 'datasets/artifacts/data_ingestion/valid')
    data_preparation.change_yaml('datasets/artifacts/data_ingestion/data.yaml')
except Exception as e:
    raise e

[2024-09-19 19:48:35,966: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-19 19:48:35,969: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-19 19:48:35,972: INFO: common: created directory at: datasets/artifacts]
[2024-09-19 19:48:35,972: INFO: common: created directory at: datasets/artifacts/data_preparation]
[2024-09-19 19:48:35,972: INFO: 1840621680: Created validation, test, and train folders]
[2024-09-19 19:48:37,514: INFO: 1840621680: Folder copied successfully from datasets/artifacts/data_ingestion/train to datasets/artifacts/data_preparation/train]
[2024-09-19 19:48:37,592: INFO: 1840621680: Folder copied successfully from datasets/artifacts/data_ingestion/test to datasets/artifacts/data_preparation/test]
[2024-09-19 19:48:37,734: INFO: 1840621680: Folder copied successfully from datasets/artifacts/data_ingestion/valid to datasets/artifacts/data_preparation/valid]
[2024-09-19 19:48:37,749: INFO: 1840621680: YAML file updated successfu